# Input variables:

In [1]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label

# set input parameter within givenr ranges
Temp = HBox([Label('Temperature [°C]'), widgets.FloatSlider(min=-20.0, max=50.0, value=21.0, step=1)])
Rad = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatSlider(min=3.22, max=7.22, value=5.22, step=0.01)])
Thick = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatSlider(min=50.0, max=100.0, value=75.6, step=0.1)])
Rr_coef = HBox([Label('Reaction rate coeffic[ent($Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatSlider(min=1.5, max=6.5, value=3.42, step=0.01)])
Dif_coef = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatSlider(min=0, max=100, value=1.48, step=0.01)])
Max_c = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatSlider(min=0.0, max=100000.0, value=51765.0, step=1)])
SOC = HBox([Label('State of charge [%]'), widgets.FloatSlider(min=0, max=100, value=100, step=1)])
C_rate = HBox([Label('Charging rate [mA/s]'), widgets.FloatSlider(min=-20.0, max=20.0, value=1.0, step=0.1)])
Vol_per = HBox([Label('Active material volume fraction [%]'), widgets.FloatSlider(min = 0.0, max=100.0, value=66.5, step=0.1)])
Area = HBox([Label('Electrode area [$m^2$]'), widgets.FloatSlider(min=0.001, max=1.0, value=0.1027, step=0.001)])
Sim_steps = HBox([Label('No. of simulation steps'), widgets.BoundedIntText(value=1000, min=1, max=10**9, disabled=False)])
Dt = HBox([Label('Time step (ns)'), widgets.FloatSlider(min=0.1, max=1000.0, value=2.0, step=0.1)]) 
Out_steps = HBox([Label('Output written every [n] steps'), widgets.BoundedIntText(value=5, min=1, max=10**9, disabled=False)])
Space_steps = HBox([Label('No. of space steps'), widgets.FloatSlider(min=2, max=1000, value=20, step=1)])


display(Temp, Rad, Thick, Rr_coef, Dif_coef, Max_c, SOC, C_rate, Vol_per, Area, Sim_steps, Dt, Out_steps, Space_steps)

In [16]:
# unbounded boxes to set paramters outside of suggested ranges if wanted
Temp = HBox([Label('Temperature [°C]'), widgets.FloatText(value=Temp.children[1].value, disabled=False)])
Rad = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatText(value=Rad.children[1].value, disabled=False)])
Thick = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatText(value=Thick.children[1].value, disabled=False)])
Rr_coef = HBox([Label('Reaction rate coeffic[ent($Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatText(value=Rr_coef.children[1].value, disabled=False)])
Dif_coef = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatText(value=Dif_coef.children[1].value, disabled=False)])
Max_c = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatText(value=Max_c.children[1].value, disabled=False)])
C_rate = HBox([Label('Charging rate [mA/s]'), widgets.FloatText(value=C_rate.children[1].value, disabled=False)])
Area = HBox([Label('Electrode area [$m^2$]'), widgets.FloatText(value=Area.children[1].value, disabled=False)])
Dt = HBox([Label('Time step (ns)'), widgets.FloatText(value=Dt.children[1].value, disabled=False)])
Space_steps = HBox([Label('No. of space steps'), widgets.FloatText(value=Space_steps.children[1].value, disabled=False)])

display(Temp, Rad, Thick, Rr_coef, Dif_coef, Max_c, C_rate, Area, Dt, Space_steps)

In [2]:
# saving the values set y the user and adjusting relevant parameters to be in SI units
Temp = Temp.children[1].value + 273.15   #K
Rad = Rad.children[1].value * 10**(-6)   #m
Thick = Thick.children[1].value * 10**(-6)   #m
Rr_coef =  Rr_coef.children[1].value
Dif_coef = Dif_coef.children[1].value * 10**(-15)
Max_c = Max_c.children[1].value
Init_c = Max_c * SOC.children[1].value / 100
C_rate = C_rate.children[1].value * 10**(-3) #A/s
Vol_per = Vol_per.children[1].value
Area = Area.children[1].value
Sim_steps = Sim_steps.children[1].value
Dt = Dt.children[1].value * 10**(-9) #m
Out_steps = Out_steps.children[1].value
Space_steps = Space_steps.children[1].value

In [3]:
#create new NetCDF file in 'writing mode' and 'NETCDF4 format'
from netCDF4 import Dataset
rootgrp = Dataset('SPM_input.nc', 'w', format='NETCDF4')
#creating dimensions for vector holding all the input variables
temp_dim = rootgrp.createDimension('temp_dim', 1)
rad_dim = rootgrp.createDimension('rad_dim', 1)
thick_dim = rootgrp.createDimension('thick_dim', 1)
rr_coef_dim = rootgrp.createDimension('rr_coef_dim', 1)
dif_coef_dim = rootgrp.createDimension('dif_coef_dim', 1)
max_c_dim = rootgrp.createDimension('max_c_dim', 1)
init_c_dim = rootgrp.createDimension('init_c_dim', 1)
c_rate_dim = rootgrp.createDimension('c_rate_dim', 1)
vol_per_dim = rootgrp.createDimension('vol_per_dim', 1)
area_dim = rootgrp.createDimension('area_dim', 1)
sim_steps_dim = rootgrp.createDimension('sim_steps_dim', 1)
dt_dim = rootgrp.createDimension('dt_dim', 1)
out_steps_dim = rootgrp.createDimension('out_steps_dim', 1)
space_steps_dim = rootgrp.createDimension('space_steps_dim', 1)
#createing variable 
temp = rootgrp.createVariable('temp', 'f8', ('temp_dim',))
rad = rootgrp.createVariable('rad', 'f8', ('rad_dim',))
thick = rootgrp.createVariable('thick', 'f8', ('thick_dim',))
rr_coef = rootgrp.createVariable('rr_coef', 'f8', ('rr_coef_dim',))
dif_coef = rootgrp.createVariable('dif_coef', 'f8', ('dif_coef_dim',))
max_c = rootgrp.createVariable('max_c', 'f8', ('max_c_dim',))
init_c = rootgrp.createVariable('init_c', 'f8', ('init_c_dim',))
c_rate = rootgrp.createVariable('c_rate', 'f8', ('c_rate_dim',))
vol_per = rootgrp.createVariable('vol_per', 'f8', ('vol_per_dim',))
area = rootgrp.createVariable('area', 'f8', ('area_dim',))
sim_steps = rootgrp.createVariable('sim_steps', 'i4', ('sim_steps_dim',))
dt = rootgrp.createVariable('dt', 'f8', ('dt_dim',))
out_steps = rootgrp.createVariable('out_steps', 'i4', ('out_steps_dim',))
space_steps = rootgrp.createVariable('space_steps', 'i4', ('space_steps_dim',))
# attributes
rootgrp.description = 'Input parameters for SMP model'
temp.description = 'Temperature'
temp.units = 'K'
rad.description = 'Mean particle radius'
rad.units = 'm'
thick.description = 'Electrode thickness'
thick.units = 'm'
rr_coef.description = 'Reaction rate coefficient'
rr_coef.units = '$Am^{-2}(m^3mol^{-1})^{1.5}$'
dif_coef.description ='Diffusion coefficient'
dif_coef.units = '$10^{-15} m^2 s^{-1}$'
max_c.description = 'Maximum lithium concentration'
max_c.units = '$mol m^{-3}$'
init_c.description = 'Initial lithium concentration'
init_c.units = '$mol m^{-3}$'
c_rate.description = 'Charging/Discharging rate'
c_rate.units = 'A/s'
vol_per.description = 'Active material volume fraction'
vol_per.units = '%'
area.description = 'Electrode area'
area.units = '$m^2$'
sim_steps.description = 'Total number of simultion steps'
sim_steps.units = 'unitless'
dt.description = 'Time step'
dt.units = 's'
out_steps.description = 'Output written every [n] number of steps'
out_steps.units = 'untiless'
space_steps.description = 'No. of space steps'
space_steps.units = 'unitless'
# writing data to input_parameters variable
temp[0] = Temp
rad[0] = Rad
thick[0] = Thick
rr_coef[0] = Rr_coef
dif_coef[0] = Dif_coef
max_c[0] = Max_c
init_c[0] = Init_c
c_rate[0] = C_rate
vol_per[0] = Vol_per
area[0] = Area
sim_steps[0] = Sim_steps
dt[0] = Dt
out_steps[0] = Out_steps
space_steps[0] = Space_steps
#closing the NetCDF file
rootgrp.close()

In [4]:
Init_c

25882.5